## Import All Necessary Dependencies

In [1]:
import time 
import math
import cv2
import mediapipe as mp
from PIL import ImageFont, ImageDraw, Image
import numpy as np

## Intialize holistic module and drawing module

In [2]:
mp_holistic=mp.solutions.holistic
mp_draw=mp.solutions.drawing_utils

## Create a function to measure the angle between fore arm and upper arm

In [3]:
def get_angle(results):
    cord1=[int(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].x*640),
          int(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y*480)]
    cord2=[int(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].x*640),
          int(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y*480)]
    cord3=[int(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].x*640),
          int(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y*480)]
    s=((cord1[0]-cord2[0])*(cord3[0]-cord2[0])) + ((cord1[1]-cord2[1])*(cord3[1]-cord2[1]) )
    a1=np.sqrt((cord1[0]-cord2[0])**2 +  (cord1[1]-cord2[1])**2)
    a2=np.sqrt((cord3[0]-cord2[0])**2 +  (cord3[1]-cord2[1])**2)
    angle= math.degrees(math.acos(s/(a1*a2)))
    return angle 

## Define a method to draw fore and upper arm

In [4]:
def draw_arm(img,results):
    cord1=[int(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].x*640),
          int(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y*480)]
    cord2=[int(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].x*640),
          int(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y*480)]
    cord3=[int(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].x*640),
          int(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y*480)]
    img=cv2.circle(img,tuple(cord1),5,(0,0,255),5)
    img=cv2.circle(img,tuple(cord2),5,(0,0,255),5)
    img=cv2.circle(img,tuple(cord3),5,(0,0,255),5)
    img=cv2.line(img,tuple(cord1),tuple(cord2),(255,0,255),3)
    img=cv2.line(img,tuple(cord2),tuple(cord3),(255,0,255),3)
    return img

## drawing the angle between fore arm and upper arm

In [5]:
def draw_angle(img,results):
    angle=get_angle(results)
    cord2=[int(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].x*640),
          int(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y*480)]
    b,g,r,a = 0,255,255,0
    fontpath = "./simsun.ttc" # the font that has this "º" symbol
    font = ImageFont.truetype(fontpath, 32)
    img_pil = Image.fromarray(img)
    draw = ImageDraw.Draw(img_pil)

    draw.text((cord2[0], cord2[1]),  f"{round(angle,2)} º", font = font, fill = (b, g, r, a))

    img = np.array(img_pil)
    return img

## Testing it on realtime

In [10]:
cap=cv2.VideoCapture(0)
s=0
holistic=mp_holistic.Holistic( min_detection_confidence=0.8,min_tracking_confidence=0.5)
while cap.isOpened():
    _,img=cap.read()
    img_rgb=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results=holistic.process(img_rgb)
    e=time.time()
    if e-s>0:
        fps=1/(e-s)
        s=e
        img=cv2.putText(img,"FPS:"+str(round(fps,2)),(15,30),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,0,255),2)
    if results.pose_landmarks:
        img=draw_arm(img,results)
        img=draw_angle(img,results)
    img=cv2.resize(img,(1000,800))
    cv2.imshow("Window",img)
    if cv2.waitKey(1) & 0xFF==ord("q"):
        break
cap.release()
cv2.destroyAllWindows()

In [7]:
## If face any issue run this shell to release your webcam manually

In [8]:
cap.release()
cv2.destroyAllWindows()